# Section ID Count and Event Data
- limited to WP:M pages

Section ID data was not limited to top-level (H2) sections during data capture, requiring post-capture processing for section ID click data.

Example of capture issue: https://en.wikipedia.org/wiki/Hepatitis#Signs_and_symptoms.
Clicks on links under "Acute hepatitis" were captured with section_id Acute_hepatitis, not Signs_and_symptoms.

Post-capture data augmentation: click event section_ids where mapped to parent H2 section headings. See [populate-section-table.ipynb](populate-section-table.ipynb) for extraction details.


In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'

## Raw section data from captured events
- Raw total count of events (by all event types) for each section ID for WP:M pages only.
- Limited to >= 3000 events.
- shows the extent of the capture issue described above


In [2]:
# "raw" section data from captured events to show extent of the capture issue described above
pm_section_events_raw_query = """
SELECT section_id, action, count(*) count
FROM 
    citationusage 
WHERE page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    AND wiki = 'enwiki'
    {}
    AND to_date(event_time) >= '{}'
    AND to_date(event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY section_id, action
ORDER BY count desc
LIMIT 100
"""
pm_section_events_raw = spark.sql(
    pm_section_events_raw_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    ))
pm_section_events_raw_pandas = pm_section_events_raw.toPandas()

In [3]:
section_pda_raw = pm_section_events_raw_pandas.copy()
# replace 'NaN' section_id with 'missing'
section_pda_raw.section_id.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered_raw = section_pda_raw.query('count>3000').copy()
# set precision before pivot
df_filtered_raw['count'] = df_filtered_raw['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered_raw.pivot(index='section_id', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
section_id,,,,
-- missing --,143490,365209,523476,None
Adverse_effects,None,4252,6343,None
Background,None,None,3496,None
Cause,None,3821,7787,None
Causes,None,10930,18686,None
Classification,None,3029,6620,None
Criminal_charges,None,3025,None,None
Diagnosis,None,7353,14525,None
Downfall,None,None,3523,None


## Count of WP:M pages by top-level (H2) section ID

In [4]:
# count of top-level (H2) section IDs for WP:M pages only
pm_sections_query = """
SELECT section_h2, count(distinct page_id) count
FROM
    ryanmax.population_wpm_sections 
GROUP BY section_h2
ORDER BY count desc, section_h2
"""
pm_sections = spark.sql(pm_sections_query)
pm_sections.toPandas().head(50)

,section_h2,count
0,References,30862
1,External_links,18460
2,See_also,13327
3,History,5666
4,Diagnosis,4654
5,Treatment,4263
6,Further_reading,3165
7,Signs_and_symptoms,2839
8,Causes,2149
9,Epidemiology,2066


## Total count of events (all types) for each top-level (H2) section ID for WP:M pages only
- Limited to >= 3000 events
- **missing** values are largely because section IDs were not recorded "if the section is the Main Section" as per [Schema:CitationUsage](https://meta.wikimedia.org/wiki/Schema:CitationUsage).
- Numbers may not exactly match "raw" data (below) because some section IDs could not be mapped to H2 section IDs (changed section ID, missing WPM page, etc.)


In [5]:
# Total count of events (by all event types) for each top-level (H2) section ID for WP:M pages only
pm_section_events_query = """
SELECT population_wpm_sections.section_h2, action, count(*) count, count(*)/{} AS daily_average
FROM 
    citationusage
    LEFT JOIN ryanmax.population_wpm_sections 
        ON 
        population_wpm_sections.page_id = citationusage.page_id 
        AND population_wpm_sections.section_id = citationusage.section_id
WHERE
    wiki = 'enwiki'
    AND citationusage.page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    {}
    AND to_date(citationusage.event_time) >= '{}'
    AND to_date(citationusage.event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY population_wpm_sections.section_h2, action
ORDER BY count desc
"""
pm_section_events = spark.sql(
    pm_section_events_query.format(
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
pm_section_events_pandas = pm_section_events.toPandas()

In [6]:
section_pda = pm_section_events_pandas.copy()
# replace 'NaN' section_h2 with 'missing'
section_pda.section_h2.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered = section_pda.query('count>3000').copy()
# set precision before pivot
df_filtered['count'] = df_filtered['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered.pivot(index='section_h2', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,144124,367446,527633,None
Adverse_effects,None,8017,14158,None
Background,None,None,4216,None
Biography,None,None,3864,None
Career,None,None,3876,None
Cause,None,8959,21897,None
Causes,None,20480,39966,None
Characteristics,None,None,3835,None
Chemistry,None,None,4100,None


#### Daily averages of above

In [7]:
df_filtered.pivot(index='section_h2', columns='action', values='daily_average')

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,4503.87500,11482.68750,16488.53125,NaN
Adverse_effects,NaN,250.53125,442.43750,NaN
Background,NaN,NaN,131.75000,NaN
Biography,NaN,NaN,120.75000,NaN
Career,NaN,NaN,121.12500,NaN
Cause,NaN,279.96875,684.28125,NaN
Causes,NaN,640.00000,1248.93750,NaN
Characteristics,NaN,NaN,119.84375,NaN
Chemistry,NaN,NaN,128.12500,NaN


In [8]:
# count of events by event type and access mode for each top-level (H2) section ID for WP:M pages only
pm_section_events_mode_query = """
SELECT population_wpm_sections.section_h2, action, mode, count(*) count
FROM 
    citationusage
    LEFT JOIN ryanmax.population_wpm_sections 
        ON 
        population_wpm_sections.page_id = citationusage.page_id 
        AND population_wpm_sections.section_id = citationusage.section_id
WHERE
    wiki = 'enwiki'
    AND citationusage.page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    {}
    AND to_date(citationusage.event_time) >= '{}'
    AND to_date(citationusage.event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY population_wpm_sections.section_h2, action, mode
ORDER BY count desc
"""
pm_section_events_mode = spark.sql(
    pm_section_events_mode_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    ))
pm_section_events_mode_pandas = pm_section_events_mode.toPandas()

## Desktop event counts for each top-level (H2) section ID for WP:M pages only
- limits above apply here as well

In [9]:
desktop_pda = pm_section_events_mode_pandas.copy()
# replace 'NaN' section_h2 with 'missing'
desktop_pda.section_h2.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered = desktop_pda.query('count>3000 & mode == "desktop"').copy()
# set precision before pivot
df_filtered['count'] = df_filtered['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered.pivot(index='section_h2', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,65483,123684,485050,None
Adverse_effects,None,None,13283,None
Background,None,None,4057,None
Biography,None,None,3729,None
Career,None,None,3744,None
Cause,None,3624,20839,None
Causes,None,7194,37401,None
Characteristics,None,None,3631,None
Chemistry,None,None,3941,None


## Mobile event counts for each top-level (H2) section ID for WP:M pages only
- limits above apply here as well

In [10]:
mobile_pda = pm_section_events_mode_pandas.copy()
# replace 'NaN' section_h2 with 'missing'
mobile_pda.section_h2.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered = mobile_pda.query('count>3000 & mode == "mobile"').copy()
# set precision before pivot
df_filtered['count'] = df_filtered['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered.pivot(index='section_h2', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,78641,243762,42583,None
Adverse_effects,None,5530,None,None
Cause,None,5335,None,None
Causes,None,13286,None,None
Diagnosis,None,8924,None,None
Epidemiology,None,4687,None,None
External_links,45292,None,None,None
Further_reading,7104,None,None,None
History,None,15350,None,None
